In [2]:
import pandas as pd
import csv
import pyltr
import numpy as np
import datetime
import re
import time
suffixes = pd.read_csv("../data/Freq_background.csv", index_col='Unnamed: 0')
sortedpopular = pd.read_csv("../data/sorted_popular_queries.csv", index_col='Unnamed: 0')
with open('../data/ngram_dict.csv', mode='r') as infile:
    reader = csv.reader(infile)
    ngram_dict = {rows[0]:rows[1] for rows in reader}
prefix_suffix_pairs_background = pd.read_csv("../data/prefix_suffix_pairs.txt")


/usr/lib/python3/dist-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [14]:
top100k = suffixes.iloc[range(100000),:]

In [11]:
from nltk import ngrams
def get_single_ngram_features(query_input, ngram_dict):
    ngram_scores = []

    for n in range(1,7):
        sixgrams = ngrams(query_input.split(), n)
        ngram_score = 0
        for grams in sixgrams:
            try:
                ngram_score = ngram_score + float(ngram_dict[' '.join(grams)])
            except:
                pass
        ngram_scores.append(ngram_score)
    return ngram_scores

def get_ngram_features(inputs, ngram_dict):
    results = [get_single_ngram_features(x, ngram_dict) for x in inputs];
    return results


In [56]:
query = input("Query: ")
original_query = input("Original query: ")

search_query = re.sub('[\W_]+',' ', query)
original_query = re.sub('[\W_]+',' ', original_query)

startt = time.time()
splitted = re.split('[\W_]+',search_query)
print(splitted)
if query[len(query)-1] == ' ':
    endterm = splitted[len(splitted)-2] + " "
else:
    endterm = splitted[len(splitted)-1]

print("Endterm -> '" + endterm + "'")

out = top100k[top100k['0'].str.startswith(endterm)].nlargest(10, 'counts')
out2 = sortedpopular[sortedpopular['0'].str.startswith(search_query)].nlargest(100, 'counts')

outcombined1 = query[:-len(endterm)] + out['0'].astype(str)
time.time() - startt
all_candidates = outcombined1.append(out2.iloc[:,0])

train_input = pd.DataFrame({
    'query': all_candidates,
    'relevant': relevant_candidate
})
  
train_input['ngram_features'] = get_ngram_features(all_candidates, ngram_dict)
print(train_input)


Query: hello
Original query: hello kitty
['hello']
Endterm -> 'hello'
                                      query  relevant  \
8281                            hello kitty      True   
16851                                 hello     False   
17184                        hello magazine     False   
23862                        hellokitty com     False   
33376                       hello kitty com     False   
42412                          hellogoodbye     False   
49195                            hellokitty     False   
52810              hello mema4chris aol com     False   
60863                         hellomagazine     False   
72194                   hellogoodbye lyrics     False   
2582                            hello kitty      True   
5028                         hello magazine     False   
8177                                  hello     False   
14402                        hellokitty com     False   
19001              hello mema4chris aol com     False   
21052             

In [12]:
input_queries = ['hello my name is jeff', 'hello my name is juan']

resultvalue = get_ngram_features(input_queries, ngram_dict)
resultvalue

[[40.770511115217005, 55.17769340395801, 28.7832171811747, 0, 0, 0],
 [41.247737418004625, 55.58315851206617, 12.390981521009305, 0, 0, 0]]